In [38]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [39]:
%sql sqlite:///../datadir/crawl-data.sqlite

'Connected: @../datadir/crawl-data.sqlite'

In [51]:
%%sql 
SELECT *
FROM javascript
LIMIT 20

 * sqlite:///../datadir/crawl-data.sqlite
Done.


id,incognito,browser_id,visit_id,extension_session_uuid,event_ordinal,page_scoped_event_ordinal,window_id,tab_id,frame_id,script_url,script_line,script_col,func_name,script_loc_eval,document_url,top_level_url,call_stack,symbol,operation,value,arguments,time_stamp
1,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,2,0,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,8010,F,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,window.navigator.vendor,get,,None,2022-11-10T16:09:07.526Z
2,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,3,1,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,7759,S,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,window.navigator.vendor,get,,None,2022-11-10T16:09:07.540Z
3,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,4,2,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,8780,G,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,window.navigator.appVersion,get,5.0 (X11),None,2022-11-10T16:09:07.540Z
4,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,5,3,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,8010,F,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,window.navigator.vendor,get,,None,2022-11-10T16:09:07.540Z
5,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,6,4,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,23979,audio,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,OfflineAudioContext.createOscillator,call,,None,2022-11-10T16:09:07.540Z
6,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,7,5,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,23998,audio,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,OscillatorNode.type,set,triangle,None,2022-11-10T16:09:07.541Z
7,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,8,6,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,24016,audio,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,OscillatorNode.frequency,get,{},None,2022-11-10T16:09:07.541Z
8,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,9,7,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,24046,audio,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,OfflineAudioContext.createDynamicsCompressor,call,,None,2022-11-10T16:09:07.541Z
9,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,10,8,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,24167,audio,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,OscillatorNode.connect,call,,"[""{}""]",2022-11-10T16:09:07.542Z
10,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,11,9,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,24178,audio,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,OfflineAudioContext.destination,get,{},None,2022-11-10T16:09:07.542Z


In [40]:
%%sql 
SELECT symbol 
FROM javascript
GROUP BY symbol

 * sqlite:///../datadir/crawl-data.sqlite
Done.


symbol
CanvasRenderingContext2D.arc
CanvasRenderingContext2D.fill
CanvasRenderingContext2D.fillRect
CanvasRenderingContext2D.fillStyle
CanvasRenderingContext2D.fillText
CanvasRenderingContext2D.font
CanvasRenderingContext2D.globalCompositeOperation
CanvasRenderingContext2D.isPointInPath
CanvasRenderingContext2D.rect
CanvasRenderingContext2D.textBaseline


In [41]:
%%sql 
SELECT script_url
FROM javascript
GROUP BY script_url

 * sqlite:///../datadir/crawl-data.sqlite
Done.


script_url
https://openfpcdn.io/fingerprintjs/v3


In [42]:
%%sql 
SELECT *
FROM javascript
WHERE symbol = "HTMLCanvasElement.toDataURL"


 * sqlite:///../datadir/crawl-data.sqlite
Done.


id,incognito,browser_id,visit_id,extension_session_uuid,event_ordinal,page_scoped_event_ordinal,window_id,tab_id,frame_id,script_url,script_line,script_col,func_name,script_loc_eval,document_url,top_level_url,call_stack,symbol,operation,value,arguments,time_stamp
43,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,44,42,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,11729,H,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,HTMLCanvasElement.toDataURL,call,,None,2022-11-10T16:09:07.552Z
44,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,45,43,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,11729,H,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,HTMLCanvasElement.toDataURL,call,,None,2022-11-10T16:09:07.552Z
61,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,62,60,1,1,0,https://openfpcdn.io/fingerprintjs/v3,8,11729,H,,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,file:///home/ndanner_plp/OpenWPM/fingerprintjsv3Demo.html,,HTMLCanvasElement.toDataURL,call,,None,2022-11-10T16:09:07.558Z


In [52]:
%%sql WITH a AS (
    SELECT script_url, SUM(script_line), SUM(script_col)
    FROM javascript
    GROUP BY script_url 
)
SELECT *
FROM a


 * sqlite:///../datadir/crawl-data.sqlite
Done.


script_url,SUM(script_line),SUM(script_col)
https://openfpcdn.io/fingerprintjs/v3,576,1708212


In [47]:
 # WHERE symbol = "HTMLCanvasElement.width" AND value > 16 AND symbol = "HTMLCanvasElement.height" AND value > 16

In [54]:
result =%sql SELECT * FROM Javascript
dataframe = result.DataFrame()

 * sqlite:///../datadir/crawl-data.sqlite
Done.


In [55]:
dataframe

,id,incognito,browser_id,visit_id,extension_session_uuid,event_ordinal,page_scoped_event_ordinal,window_id,tab_id,frame_id,...,func_name,script_loc_eval,document_url,top_level_url,call_stack,symbol,operation,value,arguments,time_stamp
0,1,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,2,0,1,1,0,...,F,,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,,window.navigator.vendor,get,,None,2022-11-10T16:09:07.526Z
1,2,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,3,1,1,1,0,...,S,,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,,window.navigator.vendor,get,,None,2022-11-10T16:09:07.540Z
2,3,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,4,2,1,1,0,...,G,,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,,window.navigator.appVersion,get,5.0 (X11),None,2022-11-10T16:09:07.540Z
3,4,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,5,3,1,1,0,...,F,,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,,window.navigator.vendor,get,,None,2022-11-10T16:09:07.540Z
4,5,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,6,4,1,1,0,...,audio,,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,,OfflineAudioContext.createOscillator,call,,None,2022-11-10T16:09:07.540Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,68,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,69,67,1,1,0,...,cookiesEnabled,,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,cookiesEnabled@https://openfpcdn.io/fingerprin...,window.document.cookie,get,cookietest=1,None,2022-11-10T16:09:07.560Z
68,69,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,70,68,1,1,0,...,cookiesEnabled,,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,cookiesEnabled@https://openfpcdn.io/fingerprin...,window.document.cookie,set,"cookietest=1; SameSite=Strict; expires=Thu, 01...",None,2022-11-10T16:09:07.560Z
69,70,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,71,69,1,1,0,...,S,,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,,window.navigator.vendor,get,,None,2022-11-10T16:09:07.560Z
70,71,0,771454105,3883045788030518,58aff3d6-f357-4f64-ac51-9efbcff99a01,72,70,1,1,0,...,G,,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,file:///home/ndanner_plp/OpenWPM/fingerprintjs...,,window.navigator.appVersion,get,5.0 (X11),None,2022-11-10T16:09:07.561Z
